In [28]:
#!/usr/bin/env python3
# import requests to ping website, bs4 to parse website with results from requests 
# import subprocess to run os commands 
# import tldextract to parse link input from user and find domain 
# import sys to kill the program if errors are found
import requests
from bs4 import BeautifulSoup as bs
import subprocess
import tldextract
import sys

In [29]:
classes = {"nytimes" : "css-1fanzo5",
"usatoday" : "gnt_ar_b_p",       
"wsj" : "wsj-snippet-body",
"foreignpolicy" : "post-content-main initial-drop-cap shares-position",
"kognity" : "content-editable",
"economist" : "article__body-text",
"fortune" : "gutenbergContent__content--1FgGp",
"bloomberg" : "body-copy-v2 fence-body"}

In [30]:
def link_check(link):
    try:
        page = requests.get(link)
        return link
    except Exception as e:
        print(e)
        print(" ")
        raise("Connection Failed")

In [31]:
def main():
    link = input("Enter link here: ")
    # Parse link and output main domain if subdomain else use domain provided in link
    try:
        link_check(link)
        if link_check == False:
            print("Please try again")
            main()
        domain = tldextract.extract(link).domain
        if domain in classes:
            run(link,domain)
        else:
            print(f"adding {domain} to sites list")
            with open("sites.txt","r+") as f:
                f.write(domain+"\n")
    except Exception as e:
        print(e)

In [32]:
def run(link,domain):
    #get page and parse it
    page = requests.get(link)
    soup = bs(page.text,'html.parser')
    # find the class of the main body paragraph
    paragraphs = str(soup.find_all(class_ = classes[domain]))
    # delete everything that is not text
    soup = bs(paragraphs)
    for script in soup(["script","style"]):
        script.extract() 
    content = soup.get_text()
    writer(link,content)

In [33]:
def writer(link,content):    
    # save results to html file and auto-close
    with open("news.html","w") as f:
        f.write(f"""<!DOCTYPE HTML PUBLIC " -//W3C//DTD HTML 4.01 Transition//EN" 
    "http://www.w3.org/TR/htm14/loose.dtd">
    <html>
      <head>
        <title>{link}</title>
        <link rel="stylesheet" href="webpage.css" type="text/css"/>
      </head>
      <body>
        <h3> The link of this webpage is {link} </h3>
        <h3> Click <a href='#'>here </a>to open the file in finder </h3>
        {content}
      </body>
    </html>""")
    # open said html file in chrome
    subprocess.run(["open", "-a", "/Applications/Google Chrome.app", "news.html"])
    main()

In [ ]:
main()